In [ ]:
import spacy
from spacy import displacy
import json


In [ ]:
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
nlp=spacy.load('en_core_web_lg')

In [ ]:
ex=nlp("Corona Virus")

In [ ]:
ex.ents

()

In [ ]:
ex2=nlp("Tesla is going to acquire U.S. startup for $6 million")

In [ ]:
spacy.displacy.render(ex2,style="ent",jupyter=True)

In [ ]:
ex2.ents

(Nicolas Tesla, U.S., $6 million)

In [ ]:
with open('/content/Corona2.json','r') as f:
  data=json.load(f)

In [ ]:
data["examples"][0].keys()

dict_keys(['id', 'content', 'metadata', 'annotations', 'classifications'])

In [ ]:
data['examples'][0]['id']

'18c2f619-f102-452f-ab81-d26f7e283ffe'

In [ ]:
data['examples'][0]['annotations'][0]

{'id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed',
 'tag_id': 'c06bd022-6ded-44a5-8d90-f17685bb85a1',
 'end': 371,
 'start': 360,
 'example_id': '18c2f619-f102-452f-ab81-d26f7e283ffe',
 'tag_name': 'Medicine',
 'value': 'Diosmectite',
 'correct': None,
 'human_annotations': [{'timestamp': '2020-03-21T00:24:32.098000Z',
   'annotator_id': 1,
   'tagged_token_id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed',
   'name': 'Ashpat123',
   'reason': 'exploration'}],
 'model_annotations': []}

In [ ]:
data["examples"][0]['content']

"While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]"

In [ ]:
training_data=[]
for ex in data['examples']:
  temp_dict={}
  temp_dict['text']=ex['content']
  temp_dict['entities']=[]
  for annotation in ex['annotations']:
    start=annotation['start']
    end = annotation['end']
    label=annotation['tag_name'].upper()
    temp_dict['entities'].append((start,end,label))
  training_data.append(temp_dict)


In [ ]:
training_data[0]['text']

"While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]"

In [ ]:
training_data[0]['entities']

In [ ]:
from spacy.tokens import DocBin
from tqdm import tqdm

nlp= spacy.blank('en')
doc_bin=DocBin()

In [ ]:
from spacy.util import filter_spans

for training_examples in tqdm(training_data):
  text=training_examples['text']
  labels=training_examples['entities']
  doc=nlp.make_doc(text)
  ents=[]
  for start,end,label in labels:
    span=doc.char_span(start,end, label=label , alignment_mode='contract')
    if span is None:
      print("Skipping Entity")
    else:
      ents.append(span)

    filtered_ents=filter_spans(ents)
    doc.ents=filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk('train.spacy')

100%|██████████| 31/31 [00:00<00:00, 159.02it/s]

Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity


In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./train.spacy --output .

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    158.43    0.00    0.00    0.00    0.00
  0     200       1106.02   3998.34   38.52   39.97   37.16    0.39
  0     400         43.01   2121.62   16.27   85.12    9.00    0.16
  1     600         52.33   1978.35   59.94   61.01   58.91    0.60
  1     800         48.83   1772.33   65.21   79.21   55.41    0.65
  1    1000         63.46   1750.21   72.08   72.69   71.48    0.72
  2    1200         65.75   1721.13   72.89   59.96   92.93    0.73
  2    1400         77.22   1912.90   68.17   67.25   69.13    0.68
  3    1600         73.77   1744.77 

In [ ]:
nlp_ner=spacy.load('/content/drive/MyDrive/model-best')

In [ ]:
doc=nlp_ner('While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers diarrhea, they do not decrease the length of illness.')
options = {"colors": colors}

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)